In [1]:
from pymer4.models import Lmer
from statsmodels.formula.api import ols
from statsmodels.stats.multitest import multipletests

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
rois_left = ['laPU', 'laCA', 'lpPU', 'lpCA']
rois_right = ['raPU', 'raCA', 'rpPU', 'rpCA']
rois_PU = ['laPU','raPU','lpPU','rpPU']
rois_CA = ['laCA','raCA','lpCA','rpCA']
rois_ant = ['laPU', 'laCA', 'raPU', 'raCA']
rois_post = ['lpPU', 'lpCA', 'rpPU', 'rpCA']

In [108]:
df = pd.read_excel('./data/Rew#1_Coef_filtered_new_KJH.xlsx')
df_n50 = pd.read_excel('./data/Rew#1_Coef_GPn50_KJH.xlsx')

In [129]:
# Check the rest of new data (data before GP71) is the same as the old data.
merged_df = pd.merge(df, df_n50, on=['subj', 'group', 'run', 'roi', 'time'], suffixes=('_1', '_n50'))
merged_df['beta_same'] = merged_df['beta_1'] == merged_df['beta_n50']
all(merged_df['beta_same'])

True

In [130]:
df_n50

,subj,group,run,roi,beta,time
0,GA01,baseline,r01,raPU,0.518611,0
1,GA01,baseline,r01,laPU,0.599993,0
2,GA01,baseline,r01,lpPU,0.151185,0
3,GA01,baseline,r01,rpPU,0.272298,0
4,GA01,baseline,r01,laCA,0.538827,0
...,...,...,...,...,...,...
1915,GP75,DLPFC_20Hz,r03,rpPU,0.614432,2
1916,GP75,DLPFC_20Hz,r03,laCA,0.917109,2
1917,GP75,DLPFC_20Hz,r03,lpCA,0.538714,2
1918,GP75,DLPFC_20Hz,r03,raCA,1.086447,2


In [131]:
df_n50['group'] = pd.Categorical(df_n50['group'], categories=['DLPFC_20Hz', 'baseline', 'DLPFC_cTBS'])

# Perform the analysis for each ROI
interaction_results_time_recoded = {}
mixedlm_result = {}

for roi in rois_left + rois_right:
    roi_data = df_n50[df_n50['roi'] == roi].dropna()
    
    # Fit the mixed effects model
    model = smf.mixedlm("beta ~ time * group", roi_data, groups=roi_data['subj'])
    mixedlm_result[roi] = model.fit()
    
    # Get the p-value of the interaction term
    print(roi)
    print(mixedlm_result[roi].df_resid)
    print(mixedlm_result[roi].pvalues['time:group[T.baseline]'])
    print(mixedlm_result[roi].pvalues['time:group[T.baseline]']*8)  # Multiple comparison correction
    print(mixedlm_result[roi].tvalues['time:group[T.baseline]'])
    print(mixedlm_result[roi].pvalues['time:group[T.DLPFC_cTBS]'])
    print(mixedlm_result[roi].pvalues['time:group[T.DLPFC_cTBS]']*8)
    print(mixedlm_result[roi].tvalues['time:group[T.DLPFC_cTBS]'])
#     print(result.df_n50_model)
    print(mixedlm_result[roi].summary())
    print('-----------------')
#     interaction_results_time_recoded[roi] = {f"time:group_recoded[T.{group}]": p_val for group, p_val in mixedlm_result.pvalues.items() if "group_recoded" in group}

#     print(mixedlm_result.summary())


laPU
183
0.09893204506665455
0.7914563605332364
-1.6500532512939659
0.0028134633296938703
0.022507706637550962
-2.9874162885954396
               Mixed Linear Model Regression Results
Model:                 MixedLM     Dependent Variable:     beta    
No. Observations:      189         Method:                 REML    
No. Groups:            63          Scale:                  0.0492  
Min. group size:       3           Log-Likelihood:         -44.9661
Max. group size:       3           Converged:              Yes     
Mean group size:       3.0                                         
-------------------------------------------------------------------
                         Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------
Intercept                 0.478    0.083  5.783 0.000  0.316  0.639
group[T.baseline]         0.124    0.102  1.210 0.226 -0.077  0.324
group[T.DLPFC_cTBS]       0.099    0.115  0.863 0.388 -0.126  0.325


-----------------------------------------------------

In [ ]:
df['group'] = pd.Categorical(df['group'], categories=['DLPFC_cTBS', 'baseline', 'M1_cTBS'])

# Perform the analysis for each ROI
interaction_results_time_recoded = {}
mixedlm_result = {}

for roi in rois_left + rois_right:
    roi_data = df[df['roi'] == roi].dropna()
    
    # Fit the mixed effects model
    model = smf.mixedlm("beta ~ time * group", roi_data, groups=roi_data['subj'])
#     model = Lmer('beta ~ time * group + (1|subj)',data =roi_data )
#     model = smf.mixedlm("beta ~ time * group_recoded", roi_data, groups=roi_data['subj'], re_formula="~time")
    mixedlm_result[roi] = model.fit()
    
    # Get the p-value of the interaction term
    print(roi)
    print(mixedlm_result[roi].df_resid)
    print(mixedlm_result[roi].pvalues['time:group[T.baseline]'])
    print(mixedlm_result[roi].pvalues['time:group[T.baseline]']*8)
    print(mixedlm_result[roi].tvalues['time:group[T.baseline]'])
    print(mixedlm_result[roi].pvalues['time:group[T.M1_cTBS]']*8)
    print(mixedlm_result[roi].tvalues['time:group[T.M1_cTBS]'])
#     print(result.df_model)
    print(mixedlm_result[roi].summary())
    print('-----------------')
#     interaction_results_time_recoded[roi] = {f"time:group_recoded[T.{group}]": p_val for group, p_val in mixedlm_result.pvalues.items() if "group_recoded" in group}

#     print(mixedlm_result.summary())


In [8]:
import seaborn as sns
from scipy.stats import f_oneway

for roi in rois_left + rois_right:
    print(roi)
    df_first_run = df[(df['run'] == 'r01') & (df['roi'] == roi)]
    print(df_first_run)
    
    # Group the data by species
    base = df[df['group'] == 'baseline']['beta']
    m1 = df[df['group'] == 'M1_cTBS']['beta']
    dlpfc = df[df['group'] == 'DLPFC_cTBS']['beta']

    # Perform one-way ANOVA
    f_stat, p_value = f_oneway(base, m1, dlpfc)
    print(f"F-statistic: {f_stat}, P-value: {p_value}")

laPU
     Unnamed: 0  subj       group  run   roi      beta  time
1             1  GA01    baseline  r01  laPU  0.599993     0
9             9  GA02    baseline  r01  laPU  0.833855     0
17           17  GA05    baseline  r01  laPU  0.946376     0
25           25  GA07    baseline  r01  laPU  0.840314     0
33           33  GA08    baseline  r01  laPU  0.041963     0
..          ...   ...         ...  ...   ...       ...   ...
473         561  GP62  DLPFC_cTBS  r01  laPU  0.420961     0
481         569  GP63     M1_cTBS  r01  laPU  0.312670     0
489         577  GP65     M1_cTBS  r01  laPU  0.211766     0
497         585  GP66  DLPFC_cTBS  r01  laPU  0.960290     0
505         593  GP67  DLPFC_cTBS  r01  laPU  0.621317     0

[64 rows x 7 columns]
F-statistic: 8.15827038020948, P-value: 0.00029897231261175934
laCA
     Unnamed: 0  subj       group  run   roi      beta  time
4             4  GA01    baseline  r01  laCA  0.538827     0
12           12  GA02    baseline  r01  laCA  0.57

In [9]:
# Subset the data to include only the first run
for roi in rois_left + rois_right:
    df_first_run = df[(df['run'] == 'r01') & (df['roi'] == roi)]

    # Fit the model
    model = ols('beta ~ group', data=df_first_run).fit()

    # Perform the ANOVA
    anova_table = sm.stats.anova_lm(model, typ=2)

    print(roi)
    print(anova_table)
    print('--------------')

laPU
            sum_sq    df         F    PR(>F)
group     0.007092   2.0  0.044343  0.956657
Residual  4.878059  61.0       NaN       NaN
--------------
laCA
            sum_sq    df         F    PR(>F)
group     0.255806   2.0  1.216573  0.303329
Residual  6.413166  61.0       NaN       NaN
--------------
lpPU
            sum_sq    df         F    PR(>F)
group     0.034784   2.0  0.323626  0.724755
Residual  3.278172  61.0       NaN       NaN
--------------
lpCA
            sum_sq    df         F    PR(>F)
group     0.070531   2.0  0.339899  0.713182
Residual  6.328969  61.0       NaN       NaN
--------------
raPU
            sum_sq    df         F    PR(>F)
group     0.045067   2.0  0.334716  0.716847
Residual  4.106603  61.0       NaN       NaN
--------------
raCA
            sum_sq    df         F    PR(>F)
group     0.026174   2.0  0.147608  0.863077
Residual  5.408274  61.0       NaN       NaN
--------------
rpPU
            sum_sq    df        F    PR(>F)
group     0.068982   

In [10]:
df[df['group'] == 'baseline']["subj"]

0       GA01
1       GA01
2       GA01
3       GA01
4       GA01
        ... 
1259    GA44
1260    GA44
1261    GA44
1262    GA44
1263    GA44
Name: subj, Length: 720, dtype: object

In [11]:
df_baseline = df[df['group']=='baseline']
unique_rois = df['roi'].unique()

for roi in unique_rois:
    roi_data = df_baseline[df_baseline['roi']==roi]
    model = smf.mixedlm("beta ~ time", roi_data, groups=roi_data["subj"])
    result = model.fit()
    
    print(f"ROI: {roi}")
    print(result.summary())
    print("\n")

ROI: raPU
        Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: beta    
No. Observations: 90      Method:             REML    
No. Groups:       30      Scale:              0.0454  
Min. group size:  3       Log-Likelihood:     -15.6891
Max. group size:  3       Converged:          Yes     
Mean group size:  3.0                                 
------------------------------------------------------
            Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------
Intercept    0.596    0.056 10.619 0.000  0.486  0.706
time        -0.093    0.028 -3.388 0.001 -0.147 -0.039
Group Var    0.057    0.108                           



ROI: laPU
        Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: beta    
No. Observations: 90      Method:             REML    
No. Groups:       30      Scale:              0.0602  
Min. group size:  3       Log-Likelihood:     -29.8122
Max. 

In [12]:
# Create a new column 'region' in the data for anterior vs posterior comparision
df["region"] = "anterior"
df.loc[df['roi'].isin(rois_post), "region"] = "posterior"

## Make a label for right and left ROIs
df["LR"] = "left"
df.loc[df['roi'].isin(rois_right), "LR"] = "right"

# Define PC for comparing putamen versus caudate nucleus
df["PC"] = "PU"
df.loc[df['roi'].isin(rois_CA), "PC"] = "CA"

# Convert 'region' to categorical
df["region"] = pd.Categorical(df["region"])

df

,Unnamed: 0,subj,group,run,roi,beta,time,region,LR,PC
0,0,GA01,baseline,r01,raPU,0.518611,0,anterior,right,PU
1,1,GA01,baseline,r01,laPU,0.599993,0,anterior,left,PU
2,2,GA01,baseline,r01,lpPU,0.151185,0,posterior,left,PU
3,3,GA01,baseline,r01,rpPU,0.272298,0,posterior,right,PU
4,4,GA01,baseline,r01,laCA,0.538827,0,anterior,left,CA
...,...,...,...,...,...,...,...,...,...,...
1531,1795,GP67,DLPFC_cTBS,r03,rpPU,0.018424,2,posterior,right,PU
1532,1796,GP67,DLPFC_cTBS,r03,laCA,0.016947,2,anterior,left,CA
1533,1797,GP67,DLPFC_cTBS,r03,lpCA,-0.135706,2,posterior,left,CA
1534,1798,GP67,DLPFC_cTBS,r03,raCA,-0.021755,2,anterior,right,CA


In [13]:
df['group'] = pd.Categorical(df['group'], categories=['DLPFC_cTBS', 'baseline', 'M1_cTBS'])

# Perform the analysis for each ROI
interaction_results_time_recoded = {}
mixedlm_result = {}

for roi in rois_left + rois_right:
    roi_data = df[df['roi'] == roi].dropna()
    
    # Fit the mixed effects model
    model = smf.mixedlm("beta ~ time * group", roi_data, groups=roi_data['subj'])
#     model = Lmer('beta ~ time * group + (1|subj)',data =roi_data )
#     model = smf.mixedlm("beta ~ time * group_recoded", roi_data, groups=roi_data['subj'], re_formula="~time")
    mixedlm_result[roi] = model.fit()
    
    # Get the p-value of the interaction term
    print(roi)
    print(mixedlm_result[roi].df_resid)
    print(mixedlm_result[roi].pvalues['time:group[T.baseline]'])
    print(mixedlm_result[roi].pvalues['time:group[T.baseline]']*8)
    print(mixedlm_result[roi].tvalues['time:group[T.baseline]'])
    print(mixedlm_result[roi].pvalues['time:group[T.M1_cTBS]']*8)
    print(mixedlm_result[roi].tvalues['time:group[T.M1_cTBS]'])
#     print(result.df_model)
    print(mixedlm_result[roi].summary())
    print('-----------------')
#     interaction_results_time_recoded[roi] = {f"time:group_recoded[T.{group}]": p_val for group, p_val in mixedlm_result.pvalues.items() if "group_recoded" in group}

#     print(mixedlm_result.summary())


laPU
186
0.07983801566227176
0.6387041252981741
1.7516267157568373
0.09218215813727862
2.5264319853504613
              Mixed Linear Model Regression Results
Model:                 MixedLM    Dependent Variable:    beta    
No. Observations:      192        Method:                REML    
No. Groups:            64         Scale:                 0.0488  
Min. group size:       3          Log-Likelihood:        -44.9340
Max. group size:       3          Converged:             Yes     
Mean group size:       3.0                                       
-----------------------------------------------------------------
                       Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------
Intercept               0.577    0.080  7.224 0.000  0.420  0.734
group[T.baseline]       0.024    0.100  0.244 0.807 -0.172  0.220
group[T.M1_cTBS]        0.029    0.113  0.259 0.796 -0.192  0.251
time                   -0.216    0.038 -5.703 0.00

In [14]:
df['group'] = pd.Categorical(df['group'], categories=['M1_cTBS', 'baseline', 'DLPFC_cTBS'])

# Perform the analysis for each ROI
interaction_results_time_recoded = {}
mixedlm_result = {}

for roi in rois_left + rois_right:
    roi_data = df[df['roi'] == roi].dropna()
    
    # Fit the mixed effects model
    model = smf.mixedlm("beta ~ time * group", roi_data, groups=roi_data['subj'])
#     model = Lmer('beta ~ time * group + (1|subj)',data =roi_data )
#     model = smf.mixedlm("beta ~ time * group_recoded", roi_data, groups=roi_data['subj'], re_formula="~time")
    mixedlm_result[roi] = model.fit()
    
    # Get the p-value of the interaction term
    print(roi)
    print(mixedlm_result[roi].df_resid)
    print(mixedlm_result[roi].pvalues['time:group[T.baseline]'])
    print(mixedlm_result[roi].pvalues['time:group[T.baseline]']*8)
    print(mixedlm_result[roi].tvalues['time:group[T.baseline]'])
    print(mixedlm_result[roi].pvalues['time:group[T.DLPFC_cTBS]']*8)
    print(mixedlm_result[roi].tvalues['time:group[T.DLPFC_cTBS]'])
#     print(result.df_model)
    print(mixedlm_result[roi].summary())
    print('-----------------')
#     interaction_results_time_recoded[roi] = {f"time:group_recoded[T.{group}]": p_val for group, p_val in mixedlm_result.pvalues.items() if "group_recoded" in group}

#     print(mixedlm_result.summary())


laPU
186
0.2700700368564807
2.1605602948518454
-1.1029012833683585
0.09218215813726961
-2.5264319853504955
               Mixed Linear Model Regression Results
Model:                 MixedLM     Dependent Variable:     beta    
No. Observations:      192         Method:                 REML    
No. Groups:            64          Scale:                  0.0488  
Min. group size:       3           Log-Likelihood:         -44.9340
Max. group size:       3           Converged:              Yes     
Mean group size:       3.0                                         
-------------------------------------------------------------------
                         Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------
Intercept                 0.606    0.080  7.589 0.000  0.450  0.763
group[T.baseline]        -0.005    0.100 -0.048 0.962 -0.201  0.191
group[T.DLPFC_cTBS]      -0.029    0.113 -0.259 0.796 -0.251  0.192
time                    

rpPU
186
0.19712176010263216
1.5769740808210573
-1.289795242818159
2.1182665705454706
-1.1151565978857414
               Mixed Linear Model Regression Results
Model:                 MixedLM      Dependent Variable:      beta  
No. Observations:      192          Method:                  REML  
No. Groups:            64           Scale:                   0.0306
Min. group size:       3            Log-Likelihood:          6.1553
Max. group size:       3            Converged:               Yes   
Mean group size:       3.0                                         
-------------------------------------------------------------------
                         Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------
Intercept                 0.283    0.058  4.923 0.000  0.171  0.396
group[T.baseline]         0.041    0.072  0.574 0.566 -0.100  0.183
group[T.DLPFC_cTBS]       0.012    0.081  0.149 0.881 -0.147  0.172
time                     

In [15]:
df_n50 = pd.read_csv('./../data/df_rew1_coef.csv')
# run_to_time = {'r01': 0, 'r02': 1, 'r03': 2}
# df_n50['time'] = df['run'].map(run_to_time)

In [16]:
df

,Unnamed: 0,subj,group,run,roi,beta,time,region,LR,PC
0,0,GA01,baseline,r01,raPU,0.518611,0,anterior,right,PU
1,1,GA01,baseline,r01,laPU,0.599993,0,anterior,left,PU
2,2,GA01,baseline,r01,lpPU,0.151185,0,posterior,left,PU
3,3,GA01,baseline,r01,rpPU,0.272298,0,posterior,right,PU
4,4,GA01,baseline,r01,laCA,0.538827,0,anterior,left,CA
...,...,...,...,...,...,...,...,...,...,...
1531,1795,GP67,DLPFC_cTBS,r03,rpPU,0.018424,2,posterior,right,PU
1532,1796,GP67,DLPFC_cTBS,r03,laCA,0.016947,2,anterior,left,CA
1533,1797,GP67,DLPFC_cTBS,r03,lpCA,-0.135706,2,posterior,left,CA
1534,1798,GP67,DLPFC_cTBS,r03,raCA,-0.021755,2,anterior,right,CA


In [17]:
df_n50

,subj,group,run,roi,roi_,beta,nvoxels
0,GA01,no_stim,r01,raPU,aPU,0.518611,135
1,GA01,no_stim,r01,laPU,aPU,0.599993,133
2,GA01,no_stim,r01,lpPU,pPU,0.151185,132
3,GA01,no_stim,r01,rpPU,pPU,0.272298,105
4,GA01,no_stim,r01,laCA,aCA,0.538827,71
...,...,...,...,...,...,...,...
1795,GP67,DLPFC_cTBS,r03,rpPU,pPU,0.018424,105
1796,GP67,DLPFC_cTBS,r03,laCA,aCA,0.016947,71
1797,GP67,DLPFC_cTBS,r03,lpCA,pCA,-0.135706,103
1798,GP67,DLPFC_cTBS,r03,raCA,aCA,-0.021755,66


In [18]:
df_n50['group'] = df_n50['group'].replace('no_stim', 'baseline')
df_n50

,subj,group,run,roi,roi_,beta,nvoxels
0,GA01,baseline,r01,raPU,aPU,0.518611,135
1,GA01,baseline,r01,laPU,aPU,0.599993,133
2,GA01,baseline,r01,lpPU,pPU,0.151185,132
3,GA01,baseline,r01,rpPU,pPU,0.272298,105
4,GA01,baseline,r01,laCA,aCA,0.538827,71
...,...,...,...,...,...,...,...
1795,GP67,DLPFC_cTBS,r03,rpPU,pPU,0.018424,105
1796,GP67,DLPFC_cTBS,r03,laCA,aCA,0.016947,71
1797,GP67,DLPFC_cTBS,r03,lpCA,pCA,-0.135706,103
1798,GP67,DLPFC_cTBS,r03,raCA,aCA,-0.021755,66


In [19]:
run_to_time = {'r01': 0, 'r02': 1, 'r03': 2}
df_n50['time'] = df_n50['run'].map(run_to_time)
df_n50

,subj,group,run,roi,roi_,beta,nvoxels,time
0,GA01,baseline,r01,raPU,aPU,0.518611,135,0
1,GA01,baseline,r01,laPU,aPU,0.599993,133,0
2,GA01,baseline,r01,lpPU,pPU,0.151185,132,0
3,GA01,baseline,r01,rpPU,pPU,0.272298,105,0
4,GA01,baseline,r01,laCA,aCA,0.538827,71,0
...,...,...,...,...,...,...,...,...
1795,GP67,DLPFC_cTBS,r03,rpPU,pPU,0.018424,105,2
1796,GP67,DLPFC_cTBS,r03,laCA,aCA,0.016947,71,2
1797,GP67,DLPFC_cTBS,r03,lpCA,pCA,-0.135706,103,2
1798,GP67,DLPFC_cTBS,r03,raCA,aCA,-0.021755,66,2


In [22]:
df_n50['group'] = pd.Categorical(df_n50['group'], categories=['DLPFC_20Hz', 'baseline', 'DLPFC_cTBS'])

# Perform the analysis for each ROI
interaction_results_time_recoded = {}
mixedlm_result = {}

for roi in rois_left + rois_right:
    roi_data = df_n50[df_n50['roi'] == roi].dropna()
    
    # Fit the mixed effects model
    model = smf.mixedlm("beta ~ time * group", roi_data, groups=roi_data['subj'])
#     model = Lmer('beta ~ time * group + (1|subj)',data =roi_data )
#     model = smf.mixedlm("beta ~ time * group_recoded", roi_data, groups=roi_data['subj'], re_formula="~time")
    mixedlm_result[roi] = model.fit()
    
    # Get the p-value of the interaction term
    print(roi)
    print(mixedlm_result[roi].df_resid)
    print(mixedlm_result[roi].pvalues['time:group[T.baseline]'])
    print(mixedlm_result[roi].pvalues['time:group[T.baseline]']*8)
    print(mixedlm_result[roi].tvalues['time:group[T.baseline]'])
    print(mixedlm_result[roi].pvalues['time:group[T.DLPFC_cTBS]'])
    print(mixedlm_result[roi].pvalues['time:group[T.DLPFC_cTBS]']*8)
    print(mixedlm_result[roi].tvalues['time:group[T.DLPFC_cTBS]'])
#     print(result.df_n50_model)
    print(mixedlm_result[roi].summary())
    print('-----------------')
#     interaction_results_time_recoded[roi] = {f"time:group_recoded[T.{group}]": p_val for group, p_val in mixedlm_result.pvalues.items() if "group_recoded" in group}

#     print(mixedlm_result.summary())


laPU
168
0.13988958700159354
1.1191166960127483
-1.476202319717153
0.0074762666141090315
0.05981013291287225
-2.674850117483869
               Mixed Linear Model Regression Results
Model:                 MixedLM     Dependent Variable:     beta    
No. Observations:      174         Method:                 REML    
No. Groups:            58          Scale:                  0.0521  
Min. group size:       3           Log-Likelihood:         -44.8421
Max. group size:       3           Converged:              Yes     
Mean group size:       3.0                                         
-------------------------------------------------------------------
                         Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------
Intercept                 0.435    0.100  4.351 0.000  0.239  0.631
group[T.baseline]         0.166    0.117  1.423 0.155 -0.063  0.395
group[T.DLPFC_cTBS]       0.142    0.128  1.107 0.269 -0.110  0.393
tim

rpPU
168
0.10374549557275449
0.8299639645820359
-1.6269604608490376
0.14460354053462693
1.1568283242770154
-1.4588603774913478
               Mixed Linear Model Regression Results
Model:                 MixedLM      Dependent Variable:      beta  
No. Observations:      174          Method:                  REML  
No. Groups:            58           Scale:                   0.0312
Min. group size:       3            Log-Likelihood:          2.6489
Max. group size:       3            Converged:               Yes   
Mean group size:       3.0                                         
-------------------------------------------------------------------
                         Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------
Intercept                 0.270    0.073  3.703 0.000  0.127  0.413
group[T.baseline]         0.055    0.085  0.645 0.519 -0.112  0.222
group[T.DLPFC_cTBS]       0.026    0.094  0.276 0.783 -0.157  0.209
time